In [ ]:
!pip install datasets
!pip install xmltodict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset

declarations_ds = load_dataset('the-french-artist/hatvp_declarations_xml', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
declarations_ds

Dataset({
    features: ['xml_sha1', 'declaration_xml'],
    num_rows: 10944
})

In [ ]:
declarations_df = declarations_ds.to_pandas()

In [ ]:
test_declaration = declarations_df.declaration_xml.to_list()[0]

In [ ]:
import xmltodict, json

o = xmltodict.parse('<e> <a>text</a> <a>text</a> </e>')
json.dumps(o) # '{"e": {"a": ["text", "text"]}}'

'{"e": {"a": ["text", "text"]}}'

In [ ]:
parsed_declaration = xmltodict.parse(test_declaration)
print(json.dumps(parsed_declaration, ensure_ascii=False, indent=2)) #pretty print option

In [ ]:
len(json.dumps(test_declaration))

12120

In [ ]:
len(json.dumps(parsed_declaration))

9668

In [ ]:
def convert_xml_to_json(row):
  parsed_declaration = xmltodict.parse(row['declaration_xml'])
  row['declaration_json'] = json.dumps(parsed_declaration, ensure_ascii=False)
  return row


declarations_ds = declarations_ds.map(convert_xml_to_json, num_proc=2)

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HF_TOKEN'))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
if True:
  declarations_ds.push_to_hub("the-french-artist/hatvp_declarations_xml_plus_json")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/346 [00:00<?, ?B/s]

## Strip B64 encoded values from the dataset.  

We find out that some very large JSONs contain B64 encoded text that is useless when embedded/used in a LLM context.  

We remove these values from JSONs.

In [ ]:
from datasets import load_dataset

declarations_ds = load_dataset('the-french-artist/hatvp_declarations_xml_plus_json', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
declarations_ds

Dataset({
    features: ['xml_sha1', 'declaration_xml', 'declaration_json'],
    num_rows: 10944
})

In [ ]:
# remove b64 values from JSON
# doesn't fail if the key is not found
def remove_b64_values_from_json(input_json):
  try:
      b64_adresse = input_json['declaration']['general']['declarant']['b64-adresse']
      input_json['declaration']['general']['declarant']['b64-adresse'] = ''
  except (KeyError, TypeError):
      pass


  try:
      b64_adresse = input_json['declaration']['general']['declarant']['b64-telephone']
      input_json['declaration']['general']['declarant']['b64-telephone'] = ''
  except (KeyError, TypeError):
      pass


  try:
      b64_adresse = input_json['declaration']['attachedFiles']['attachedFiles']['base64EncodedContent']
      input_json['declaration']['attachedFiles']['attachedFiles']['base64EncodedContent'] = ''
  except (KeyError, TypeError):
      pass

  return input_json

In [ ]:
import json

def remove_b64_from_sample(row):
  clean_json = remove_b64_values_from_json(
      json.loads(
          row['declaration_json']
          )
  )
  row['declaration_json'] = json.dumps(clean_json, ensure_ascii=False)
  return row

In [ ]:
declarations_ds = declarations_ds.map(remove_b64_from_sample, num_proc=2)

Map (num_proc=2):   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
declarations_ds.push_to_hub("the-french-artist/hatvp_declarations_xml_plus_json")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/393 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/the-french-artist/hatvp_declarations_xml_plus_json/commit/accde89764d83252842979d930f9c6073ebc7552', commit_message='Upload dataset', commit_description='', oid='accde89764d83252842979d930f9c6073ebc7552', pr_url=None, pr_revision=None, pr_num=None)